In [1]:
import pandas as pd
import numpy as np


from sqlalchemy import create_engine


mysql_properties_prod = {  # mysql 生产环境配置
    "host": "10.129.41.147",
    "port": "4918",
    "url": "jdbc:mysql://10.129.41.147:4918/ck",
    "user": "dmp_sys_guest",
    "password": "fTe^3hy*dO_363",
    "driver": "com.mysql.cj.jdbc.Driver",
    "database": "ck"
}

mysql_engine_prod = create_engine(
    f"mysql+pymysql://{mysql_properties_prod['user']}:{mysql_properties_prod['password']}" +
    f"@{mysql_properties_prod['host']}:{mysql_properties_prod['port']}/{mysql_properties_prod['database']}?charset=utf8"
)


# 整合etl_map_all表

将sdec所有系列的配置规整到all一个系列中

In [3]:
# 对etl_map_all数据，分割pid
# TODO: 发现e系列 中冷后温度，pid数量与单位数量不匹配
# 读取etl_map_all
etl_map_df = pd.read_sql(
    sql=f"SELECT * FROM dmp.etl_map_all WHERE data_source = 'snat'",
    con=mysql_engine_prod
)

etl_map_flatten_list = []
for _, row in etl_map_df.iterrows():
    if row['signal_code'] is None:
        continue

    for pid_id, pid in enumerate(row['signal_code'].split(';')):
        if not row['signal_unit'] is None:
            units = row['signal_unit'].split(';')
            etl_map_flatten_list.append([row['target_desc'], row['target_name'], row['target_unit'], pid,
                                         row['hos_series'], units[pid_id]])
        else:
            etl_map_flatten_list.append([row['target_desc'], row['target_name'], row['target_unit'], pid,
                                         row['hos_series'], None])

etl_map_flatten_df = pd.DataFrame(etl_map_flatten_list, columns=['target_desc', 'target_name', 'target_unit', 'pid',
                                                                 'hos_series', 'signal_unit'])
etl_map_flatten_df

,target_desc,target_name,target_unit,pid,hos_series,signal_unit
0,gps经度,lng,°,longitude,E,°
1,gps纬度,lat,°,latitude,E,°
2,大气压力,atmospheric_pressure,kpa,SAE_J000108,E,kpa
3,大气温度,atmospheric_temperature,℃,SAE_J000171,E,℃
4,车速,speed,km/h,SAE_J000084,E,km/h
...,...,...,...,...,...,...
237,行车再生请求,Driving_regeneration,None,C81_U027400,R,None
238,距上次再生完成发动机运行时间,rgn_interval,h,MD1_U115C00,R,s
239,距上次再生完成发动机运行时间,rgn_interval,h,C81_U01F200,R,s
240,再生请求标志,PFltRgn_stDem,None,MD1_U115500,R,None


In [16]:
def join_unit(unit_lst):
    without_none_unit_list = [i for i in unit_lst if i is not None]
    if len(without_none_unit_list) == 0:
        return None
    elif len(without_none_unit_list) == 1:
        return without_none_unit_list[0]
    else:
        return ';'.join(without_none_unit_list)

elt_target_name_gdf = etl_map_flatten_df.groupby("target_name")
elt_all_series_list = []
for target_name, group in elt_target_name_gdf:
    each_target_dict = {}
    unique_pid_unit_df = group.loc[:, ['pid', 'signal_unit']].drop_duplicates()
    
    # 拼接pid与单位
    pids = ';'.join(unique_pid_unit_df['pid'].tolist())
    units = join_unit(unique_pid_unit_df['signal_unit'].tolist())
    each_target_dict['target_desc'] = group['target_desc'].values[0]
    each_target_dict['target_name'] = target_name
    each_target_dict['signal_code'] = pids
    each_target_dict['target_unit'] = group['target_unit'].values[0]
    each_target_dict['signal_unit'] = units
    each_target_dict['data_source'] = 'snat'
    each_target_dict['platname'] = '大通'
    each_target_dict['hos_series'] = 'all'
    each_target_dict['data_type'] = 'gc'
    elt_all_series_list.append(each_target_dict)
elt_all_series_df = pd.DataFrame(elt_all_series_list)
elt_all_series_df.to_sql(
    name='etl_map_all',
    schema='dmp',
    if_exists='append',
    index=False,
    con=mysql_engine_prod
)
elt_all_series_df

,target_desc,target_name,signal_code,target_unit,signal_unit,data_source,platname,hos_series,data_type
0,行车再生请求,Driving_regeneration,DCW_U010702;MD1_U115500;C81_U027400,None,None,snat,大通,all,gc
1,后喷油量,InjCrv_qPoI3Set,DCW_U010515,mm3/st,mm3/st,snat,大通,all,gc
2,循环喷油量,InjCtl_qCurr,MD1_U106600;SAE_J516099,mg/hub,mg/hub;mg/hub,snat,大通,all,gc
3,再生间隔里程,PFltRgn_lSnceRgn,MD1_U114D00;DCW_U01070E;C81_U01EF00,km,m;km;m,snat,大通,all,gc
4,再生请求标志,PFltRgn_stDem,MD1_U115500;C81_U027400,None,None,snat,大通,all,gc
5,驻车请求,Parking_regeneration,DCW_U010706,None,None,snat,大通,all,gc
6,油门踏板位置,accpp,SAE_J000091,%,%,snat,大通,all,gc
7,实际发动机扭矩百分比,actual_engine_tq_per,SAE_J000513,%,%,snat,大通,all,gc
8,大气压力,atmospheric_pressure,SAE_J000108,kpa,kpa,snat,大通,all,gc
9,大气温度,atmospheric_temperature,SAE_J000171,℃,℃,snat,大通,all,gc
